## In this notebook, we plot various candle prices, and the moving average

In [35]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [36]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [8, 16, 32, 64, 128, 256]

df = pd.read_pickle(utils.get_hist_data_filename(pair, granularity))
#df.head()
#df.describe()
#df.info()
#df.columns

In [37]:
# convert object(strings) types to integer types for data manipulation
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]

df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [38]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [39]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
    
df_ma.dropna(inplace=True)

In [40]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_8,MA_16,MA_32,MA_64,MA_128,MA_256
255,2024-06-27T02:00:00.000000000Z,1.06889,1.06916,1.06882,1.06894,1.068235,1.068219,1.069379,1.071020,1.071061,1.072707
256,2024-06-27T03:00:00.000000000Z,1.06893,1.06925,1.06882,1.06908,1.068356,1.068198,1.069300,1.070949,1.071019,1.072678
257,2024-06-27T04:00:00.000000000Z,1.06906,1.06948,1.06906,1.06944,1.068535,1.068306,1.069238,1.070886,1.070981,1.072627
258,2024-06-27T05:00:00.000000000Z,1.06943,1.06973,1.06921,1.06958,1.068728,1.068436,1.069182,1.070813,1.070942,1.072572
259,2024-06-27T06:00:00.000000000Z,1.06954,1.06990,1.06911,1.06930,1.068881,1.068516,1.069122,1.070755,1.070901,1.072511


In [41]:
# plot last 100 candles
df_plot = df_ma.iloc[-400:].copy()

In [42]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, 
    open=df_plot.mid_o, 
    high=df_plot.mid_h, 
    low=df_plot.mid_l, 
    close=df_plot.mid_c,
    line=dict(width=1),
    opacity=1,
    increasing_fillcolor="#24A06B",
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color="#2EC886",
    decreasing_line_color="#FF3A4C"
))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(
        x=df_plot.time,
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
))
fig.update_layout(
    width=1000, height=400,
    font=dict(size=10, color="#e1e1e1"),
    margin=dict(l=10, r=10, b=10, t=10),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e"
)
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True, fixedrange=True, #rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()